# Serialization Walkthrough

In [73]:
from datetime import datetime
from pydantic import BaseModel, Json
from typing import Any, List

Now lets define a class that inherits from the BaseModel

In [74]:
class Location(BaseModel):
    lat: float
    lon: float

class Meeting(BaseModel):
    when: datetime
    where: str = 'Satori Athens Offices'
    why: str = 'Party'
    location: Location

class ListJson(BaseModel):
    jsn_list: List[Json[Any]]

In [75]:
location = Location(lat=37.9838, lon=23.7275)
meet = Meeting(when='2020-01-01T12:00', where='Satori Offices', location=location)

In [96]:
json_list = ListJson(jsn_list=['{"a": 1}', '{"b": 2}'])

## To dict

In [79]:
# Simple as that to convert it to a python dict
value = dict(location)

## model_dump & model_dump_....

In [85]:
meet.model_dump(exclude_unset=True)

{'when': datetime.datetime(2020, 1, 1, 12, 0),
 'where': 'Satori Offices',
 'location': {'lat': 37.9838, 'lon': 23.7275}}

In [64]:
# Here we get the why field as well but it was not set
meet.model_dump()

{'when': datetime.datetime(2020, 1, 1, 12, 0),
 'where': 'home',
 'why': 'Party',
 'location': {'lat': 37.9838, 'lon': 23.7275}}

In [86]:
# If you want to present the data directly into the app
print(meet.model_dump_json())

{"when":"2020-01-01T12:00:00","where":"Satori Offices","why":"Party","location":{"lat":37.9838,"lon":23.7275}}


In [87]:
print(meet.model_dump_json(indent=2))

{
  "when": "2020-01-01T12:00:00",
  "where": "Satori Offices",
  "why": "Party",
  "location": {
    "lat": 37.9838,
    "lon": 23.7275
  }
}


In [89]:
# for field_name, field_value in model also works in pydantic but sub-models will not be converted to dictionaries
for field_name, field_value in meet:
    print(field_name, "-->", field_value)

when --> 2020-01-01 12:00:00
where --> Satori Offices
why --> Party
location --> lat=37.9838 lon=23.7275


In [92]:
# Make the call appropriate for JSON serialization to be sent in the database
meet.model_dump(exclude={'where'}, mode='json')

{'when': '2020-01-01T12:00:00',
 'why': 'Party',
 'location': {'lat': 37.9838, 'lon': 23.7275}}

In [94]:
# Or directly select what will be sent with include
print(meet.model_dump_json(include={'when', 'where'}))

{"when":"2020-01-01T12:00:00","where":"Satori Offices"}


In [98]:
# Serialize the json list
json_list.model_dump()

{'jsn_list': [{'a': 1}, {'b': 2}]}

In [99]:
# Indicates that the serialization process should preserve the format of the original input data as closely as possible.
json_list.model_dump(round_trip=True)

{'jsn_list': ['{"a":1}', '{"b":2}']}

## Custom Serializers

In [100]:
# All the available custom serializers in pydantic
from pydantic import BaseModel, field_serializer, model_serializer, PlainSerializer, WrapSerializer

from typing import Set, Literal, List, Any, Dict
from typing_extensions import Annotated
import numpy as np
from datetime import datetime, timezone

### field_serializer

Decorator that enables **custom field serialization**.

In [101]:
class Satorian(BaseModel):
    name: str = 'John Doe'
    favouite_drinks: Set[str]

    @field_serializer('favouite_drinks')
    def serialize_favourite_drinks_with_alphabetical_order(favouite_drinks: Set[str]):
        return sorted(favouite_drinks)

employee = Satorian(favouite_drinks={'Old Fashioned', 'Aperol Spritz', 'Negroni', 'Tom Collins', 'Martini','Margarita'})
print(employee.model_dump_json(indent=4))

{
    "name": "John Doe",
    "favouite_drinks": [
        "Aperol Spritz",
        "Margarita",
        "Martini",
        "Negroni",
        "Old Fashioned",
        "Tom Collins"
    ]
}


### model_serializer

Decorator that enables **custom model serialization**. 
This is useful when a model need to be serialized in a customized manner, allowing for flexibility beyond just specific fields.

In [102]:
class PizzaModel(BaseModel):
    unit: Literal['diameter', 'circumference', 'radius', 'area']
    value: float

    @model_serializer()
    def serialize_pizza_model(self):
        if self.unit == 'circumference':
            return {'unit': 'diameter', 'value': self.value / np.pi}
        if self.unit == 'radius':
            return {'unit': 'diameter', 'value': self.value * 2}
        if self.unit == 'area':
            return {'unit': 'diameter', 'value': 2 * np.sqrt(self.value / np.pi)}
        return {'unit': self.unit, 'value': self.value}

In [103]:
pizza = PizzaModel(unit='area', value=32)
print(pizza.model_dump_json(indent=2))

{
  "unit": "diameter",
  "value": 6.383076486422923
}


### PlainSerializer 

Plain serializers is a wrapper that **use a function to modify the output** of serialization.

In [104]:
CustomString = Annotated[List, PlainSerializer(lambda x: ', '.join(x), return_type=str)]

class SatoriansOrder(BaseModel):
    order_list: CustomString

order = SatoriansOrder(order_list=['Old Fashioned', 'Aperol Spritz', 'Negroni', 'Tom Collins', 'Pizza for newcomers', 'Feyrouz'])
print(order.model_dump_json(indent=2))

{
  "order_list": "Old Fashioned, Aperol Spritz, Negroni, Tom Collins, Pizza for newcomers, Feyrouz"
}


### WrapSerializer 

Wrap serializer **receive the raw input along with a handler function that applies the standard serialization logic**, and can modify the resulting value before returning it as the final output of serialization.

In [105]:
class SatoriEventDatetime(BaseModel):
    start: datetime
    end: datetime

def convert_to_utc(value: Any, handler, info) -> Dict[str, datetime]:
    partial_result = handler(value, info)
    if info.mode == 'json':
        return {
            k: datetime.fromisoformat(v).astimezone(timezone.utc) for k, v in partial_result.items()
        }
    return {k: v.astimezone(timezone.utc) for k, v in partial_result.items()}

In [106]:
UTCEventDatetime = Annotated[SatoriEventDatetime, WrapSerializer(convert_to_utc)]

class EventModel(BaseModel):
    event_datetime: UTCEventDatetime

satori_event_time = SatoriEventDatetime(
    start='2024-04-04T15:00:00+02:00', end='2024-04-03T17:00:00+02:00'
)
event = EventModel(event_datetime=satori_event_time)
print(event.model_dump())

{'event_datetime': {'start': datetime.datetime(2024, 4, 4, 13, 0, tzinfo=datetime.timezone.utc), 'end': datetime.datetime(2024, 4, 3, 15, 0, tzinfo=datetime.timezone.utc)}}


In [107]:
print("Event time in UTC so Mark will know:")
print(event.model_dump_json(indent=2))

Event time in UTC so mark will know:
{
  "event_datetime": {
    "start": "2024-04-04T13:00:00Z",
    "end": "2024-04-03T15:00:00Z"
  }
}


## Pickle works!

In [118]:
import pickle

In [119]:
data = pickle.dumps(satori_event_time)
data

b'\x80\x04\x95\x0f\x01\x00\x00\x00\x00\x00\x00\x8c\x08__main__\x94\x8c\x13SatoriEventDatetime\x94\x93\x94)\x81\x94}\x94(\x8c\x08__dict__\x94}\x94(\x8c\x05start\x94\x8c\x08datetime\x94\x8c\x08datetime\x94\x93\x94C\n\x07\xe8\x04\x04\x0f\x00\x00\x00\x00\x00\x94\x8c\x1cpydantic_core._pydantic_core\x94\x8c\x06TzInfo\x94\x93\x94M \x1c\x85\x94R\x94\x86\x94R\x94\x8c\x03end\x94h\nC\n\x07\xe8\x04\x03\x11\x00\x00\x00\x00\x00\x94h\x0eM \x1c\x85\x94R\x94\x86\x94R\x94u\x8c\x12__pydantic_extra__\x94N\x8c\x17__pydantic_fields_set__\x94\x8f\x94(h\x13h\x07\x90\x8c\x14__pydantic_private__\x94Nub.'

In [120]:
retreived_event_data = pickle.loads(data)
print(retreived_event_data.model_dump_json(indent=2))

{
  "start": "2024-04-04T15:00:00+02:00",
  "end": "2024-04-03T17:00:00+02:00"
}
